In [3]:
### load arrs, pgv4 new

import jax
import jax.numpy as jnp
from jax import jit
import jax.random as rnd
import numpy as np

np.set_printoptions(threshold=1000, linewidth=200)
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.patches import Ellipse
import os
import functools

def load_large_outputs(str_):
    path_ = os.path.dirname(os.getcwd()) + '/sc_project/large_outputs/' # '/sc_project/test_data/' # Path(__file__).resolve().parents[1]
    with open(path_+str_,'rb') as file_:
        # param = pickle.load(file_)
        param_ = jnp.load(file_,allow_pickle=True)
    return param_

def freeze_array(BA, P):
    P2 = np.zeros_like(P)
    P2[:, 0] = P[:, 0]
    for t in range(1, P.shape[1]):
        if BA[t] == 0:
            P2[:, t] = P[:, t]  # Update to current value in P
        else:
            P2[:, t] = P2[:, t - 1]  # Hold previous value
    return P2

def compare_arrays(pos_arr, pos_plan_arr):
    return np.where(pos_arr == pos_plan_arr, 0, 1)

file_ =  'outer_loop_pg_new_v4f_23_10-231650.pkl'
other = load_large_outputs(file_)
(r_arr,rp_arr,sample_arr,mask_arr,pos_plan_arr,pos_arr,dot_arr,policy_arr,hs_arr,hv_arr) = other
print('r_arr.shape=',r_arr.shape,'rp_arr.shape=',rp_arr.shape,'sample_arr.shape=',sample_arr.shape,'mask_arr.shape=',mask_arr.shape,'\n','pos_plan_arr.shape=',pos_plan_arr.shape,'pos_arr.shape=',pos_arr.shape,'dot_arr.shape=',dot_arr.shape,'policy_arr.shape=',policy_arr[0].shape,'\n','hs_arr=',hs_arr.shape,'hv_arr=',hv_arr.shape)
print('r_tot=',np.mean(np.sum(r_arr,axis=1)))

r_arr.shape= (1000, 60) rp_arr.shape= (1000, 60) sample_arr.shape= (1000, 60) mask_arr.shape= (1000, 60) 
 pos_plan_arr.shape= (1000, 61, 2) pos_arr.shape= (1000, 61, 2) dot_arr.shape= (1000, 61, 2) policy_arr.shape= (1000, 60, 81) 
 hs_arr= (1000, 60, 100) hv_arr= (1000, 60, 300)
r_tot= 3.9440958


In [ ]:
# binned sort
%matplotlib inline

# Calculate dot speeds
dot_diffs = np.diff(dot_arr, axis=1)
dot_speeds = np.linalg.norm(dot_diffs, axis=2)
mean_dot_speeds = np.mean(dot_speeds, axis=1)

# Define speed bins
speed_bins = np.linspace(np.min(mean_dot_speeds), np.max(mean_dot_speeds), num=5)  # Assuming 4 bins
speed_bin_centers = (speed_bins[:-1] + speed_bins[1:]) / 2

# Loop through speed bins
for i in range(len(speed_bins) - 1):
    mask = (mean_dot_speeds >= speed_bins[i]) & (mean_dot_speeds < speed_bins[i + 1])
    
    masked_indices = np.where(mask)[0]  # Keep track of original indices
    masked_decisions = sample_arr[mask]
    masked_rewards = r_arr[mask]
    
    total_rewards = np.sum(masked_rewards, axis=1)
    planning_rates = np.mean(masked_decisions, axis=1)
    product_scores = total_rewards * planning_rates

    # Get the sorted indices in descending order
    reward_indices = np.argsort(total_rewards)[::-1]
    planning_indices = np.argsort(planning_rates)[::-1]
    product_indices = np.argsort(product_scores)[::-1]

    # Top 5 indices for rewards and planning rates
    top_reward_indices = reward_indices[:5]
    top_planning_indices = planning_indices[:50]
    top_product_indices = product_indices[:10]
    
    # Map back to original indices
    original_top_reward_indices = masked_indices[top_reward_indices]
    original_top_planning_indices = masked_indices[top_planning_indices]
    original_top_product_indices = masked_indices[top_product_indices]
    
    # Print the results
    print("top rewards for speed bin", i+1)
    for idx, original_idx in enumerate(original_top_reward_indices):
        print(f" Index: {original_idx}, Reward: {total_rewards[top_reward_indices[idx]]:.2f}")

    print("\n top plan rates for speed bin", i+1)
    for idx, original_idx in enumerate(original_top_planning_indices):
        print(f"Index: {original_idx}, Planning Rate: {planning_rates[top_planning_indices[idx]]:.2f}, reward: {np.sum(r_arr[original_idx,:]):.2f}")

    print("\n top Product Scores for speed bin", i+1)
    for idx, original_idx in enumerate(original_top_product_indices):
        print(f"Index: {original_idx}, Score: {product_scores[top_product_indices[idx]]:.2f}")
    print('----------------------')

In [11]:
# PCA using SVD; get low-dim trajectories of top K PC's

import numpy as np

K = 3

def pca_svd_method(data, K):
    # Reshape the data
    num_samples, T, H = data.shape
    reshaped_data = data.reshape(num_samples*T, H)
    
    # Center the data
    mean_data = np.mean(reshaped_data, axis=0)
    centered_data = reshaped_data - mean_data
    
    # Compute the SVD
    U, S, V = np.linalg.svd(centered_data, full_matrices=False)
    
    # Get the top K columns of V
    V_topK = V.T[:, :K]
    
    # Project data onto the top K PCs
    projected_data = centered_data @ V_topK
    
    # Reshape the projected data back to [1000, 60, K]
    projected_data = projected_data.reshape(num_samples, T, K)
    
    return projected_data

hs_pc_s = pca_svd_method(hs_arr, K)
hv_pc_s = pca_svd_method(hv_arr, K)

print(hs_pc_s.shape)  # Should print (1000, 60, 3)
print(hv_pc_s.shape)  # Should print (1000, 60, 3)

(1000, 60, 3)
(1000, 60, 3)


In [9]:
# PCA using covar matrix; get low-dim trajectories of top K PC's

import numpy as np

K = 3

def pca_covariance_method(data, K):
    # Reshape the data to stack all time points across all trials
    num_samples, T, H = data.shape
    reshaped_data = data.reshape(num_samples*T, H)

    # Subtract the overall mean across trials and time points
    mean_data = np.mean(reshaped_data, axis=0)
    centered_data = reshaped_data - mean_data
    
    # Calculate covariance matrix
    cov_matrix = np.cov(centered_data, rowvar=False)
    
    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    
    # Sorting eigenvectors based on eigenvalues
    idx = eigenvalues.argsort()[::-1]  # Sort in descending order
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # Take the top K eigenvectors
    top_eigenvectors = eigenvectors[:, :K]
    
    # Project data onto top K eigenvectors to get principal components
    projected_data = np.array([np.dot(data[i], top_eigenvectors) for i in range(data.shape[0])])
    
    return projected_data


hs_pc_c = pca_covariance_method(hs_arr, K)
hv_pc_c = pca_covariance_method(hv_arr, K)


In [ ]:
# plot specified trajectories; plans in black

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def plot_trajectories(fig, data, sample_arr, n_list):
    colormaps = ['PiYG', 'PRGn', 'PuOr', 'RdBu']
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for idx, n in enumerate(n_list):
        traj = data[n, :, :3]
        length = traj.shape[0]
        
        # Compute transitions in sample_arr for current trajectory
        transitions = np.where(np.diff(sample_arr[n, :]) != 0)[0]
        print('tr=',transitions)
        
        # Add start or end point if the trajectory starts or ends with '1'
        if sample_arr[n, 0] == 1:
            transitions = np.insert(transitions, 0, -1)
        if sample_arr[n, -1] == 1:
            transitions = np.append(transitions, length-2)
        
        # Create color array based on sample_arr
        colors = np.array([plt.get_cmap(colormaps[idx])(i) if sample_arr[n, i] == 0 else (0, 0, 0, 1) for i in range(length)])
        
        segments_3d = [traj[i:i+2] for i in range(length-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=colors[:-1])
        lc_2d = LineCollection(segments_2d, colors=colors[:-1])
        
        ax1.add_collection(lc_3d)
        ax2.add_collection(lc_2d)

        # Add dots for transitions
        if sample_arr[n, 0] == 1:
            ax1.scatter(*traj[0, :3], color='black', s=10)
            ax2.scatter(*traj[0, :2], color='black', s=10)

        if sample_arr[n, -1] == 1:
            ax1.scatter(*traj[-1, :3], color='black', s=10)
            ax2.scatter(*traj[-1, :2], color='black', s=10)

        for t in transitions:
            ax1.scatter(*traj[t+1, :3], color='black', s=10)
            ax2.scatter(*traj[t+1, :2], color='black', s=10)

    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())

    ax1.set_title('3D Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Trajectories')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

fig = plt.figure(figsize=(12, 6))
n_list = [796, 186, 358]
# [884, 87, 20] # R2
# [335,796,479] # R1
plot_trajectories(fig, hs_pc_s, sample_arr, n_list)

In [42]:
# trajectories binned by dot vector (top T)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]  # Omitting the last angle to avoid overlap with 0
    return np.column_stack((np.cos(angles), np.sin(angles)))

# def has_duplicates(array_2d):
#     flat_list = array_2d.flatten().tolist()
#     unique_set = set(flat_list)
#     return len(unique_set) < len(flat_list)

def generate_polar_angles(N):
    step = 360 / N
    angles = [(i * step) % 360 for i in range(N)]
    return [f"{int(angle)}°" if angle <= 180 else f"{int(angle - 360)}°" for angle in angles]

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def get_top_rewarded_trials_new(dot_arr, r_arr, T, N):
    bins = generate_bin_vectors(N)
    normalized_dot_dirs = compute_dot_direction(dot_arr)

    # Assign each dot to a bin
    bin_assignments = []
    for dot in normalized_dot_dirs:
        cosine_similarity = dot @ np.array(bins).T
        assigned_bin = np.argmax(cosine_similarity)
        bin_assignments.append(assigned_bin)

    # Organize trials by their assigned bin and then sort them by total reward
    total_rewards = np.sum(r_arr, axis=1)
    bin_lists = [[] for _ in range(N)]
    for i, assigned_bin in enumerate(bin_assignments):
        bin_lists[assigned_bin].append(i)
    
    # Sort trials in each bin by reward and select top T trials
    top_trials = np.zeros((T, N), dtype=int)
    for idx, trial_list in enumerate(bin_lists):
        sorted_trials = sorted(trial_list, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    return top_trials

def plot_trajectories(fig, data, top_trials, N):
    # Define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T): # Transpose to iterate over trials per bin
        for trial_idx in trials:
            traj = data[trial_idx, :, :3]
            segments_3d = [traj[i:i+2] for i in range(traj.shape[0]-1)]
            segments_2d = [s[:, :2] for s in segments_3d]
            
            lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
            lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
            
            ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())

    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Trajectories')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

def plot_avg_trajectories(fig, data, top_trials, N):
    # Define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T): 
        # Calculate the average trajectory for the trials in this bin
        avg_traj = np.mean(data[trials], axis=0)
        
        segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
        lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
        
        ax1.add_collection(lc_3d)
        ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    # Create a legend with polar angles corresponding to the bins
    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Average Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Average Trajectories')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

N = 8
T = 5

# Assuming hs_pc_s data is available
fig = plt.figure(figsize=(12, 6))
top_rewarded_trials = get_top_rewarded_trials_new(dot_arr, r_arr, T, N)
print('trt=',top_rewarded_trials.shape)
# plot_avg_trajectories(fig, hs_pc_s, top_rewarded_trials, N)
plot_trajectories(fig, hs_pc_s, top_rewarded_trials, N)


trt= (5, 8)


KeyboardInterrupt: 

In [41]:
# trajectories binned by dot vector (random)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]  # Omitting the last angle to avoid overlap with 0
    return np.column_stack((np.cos(angles), np.sin(angles)))

# def has_duplicates(array_2d):
#     flat_list = array_2d.flatten().tolist()
#     unique_set = set(flat_list)
#     return len(unique_set) < len(flat_list)

def generate_polar_angles(N):
    step = 360 / N
    angles = [(i * step) % 360 for i in range(N)]
    return [f"{int(angle)}°" if angle <= 180 else f"{int(angle - 360)}°" for angle in angles]

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def get_binned_trials(dot_arr, N):
    bins = generate_bin_vectors(N)
    normalized_dot_dirs = compute_dot_direction(dot_arr)

    # Assign each dot to a bin
    bin_assignments = []
    for dot in normalized_dot_dirs:
        cosine_similarity = dot @ np.array(bins).T
        assigned_bin = np.argmax(cosine_similarity)
        bin_assignments.append(assigned_bin)

    # Organize trials by their assigned bin
    bin_lists = [[] for _ in range(N)]
    for i, assigned_bin in enumerate(bin_assignments):
        bin_lists[assigned_bin].append(i)

    # Find the bin with the maximum number of trials
    max_bin_length = max(len(trial_list) for trial_list in bin_lists)
    
    # Preallocate an array filled with nans
    binned_trials = np.full((max_bin_length, N), np.nan)
    
    # Fill the array with trial indices for each bin
    for idx, trial_list in enumerate(bin_lists):
        binned_trials[:len(trial_list), idx] = trial_list

    return binned_trials

def plot_trajectories(fig, data, top_trials, N, T):
    # Define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T): # Transpose to iterate over trials per bin
        valid_trials = trials[~np.isnan(trials)].astype(int)  # Remove nan values and convert to int
        
        # Select T random trials from valid trials
        selected_trials = np.random.choice(valid_trials, min(T, len(valid_trials)), replace=False)
        
        for trial_idx in selected_trials:
            traj = data[trial_idx, :, :3]
            segments_3d = [traj[i:i+2] for i in range(traj.shape[0]-1)]
            segments_2d = [s[:, 1:3] for s in segments_3d]
            
            lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
            lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
            
            ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 1].min(), all_data[:, 1].max())
    ax2.set_ylim(all_data[:, 2].min(), all_data[:, 2].max())

    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Trajectories')
    ax2.set_xlabel('PC2')
    ax2.set_ylabel('PC3')
    
    plt.show()

def plot_avg_trajectories(fig, data, top_trials, N):
    # Define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T):
        valid_trials = trials[~np.isnan(trials)].astype(int)  # Remove nan values and convert to int
        # Calculate the average trajectory for the trials in this bin
        avg_traj = np.mean(data[valid_trials], axis=0)
        
        segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
        segments_2d = [s[:, 1:3] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
        lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
        
        ax1.add_collection(lc_3d)
        ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 1].min(), all_data[:, 1].max())
    ax2.set_ylim(all_data[:, 2].min(), all_data[:, 2].max())
    
    # Create a legend with polar angles corresponding to the bins
    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Average Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Average Trajectories')
    ax2.set_xlabel('PC2')
    ax2.set_ylabel('PC3')
    
    plt.show()

N = 8
T = 100

# Assuming hs_pc_s data is available
fig = plt.figure(figsize=(12, 6))
# top_rewarded_trials = get_random_trials(dot_arr, T, N)
top_rewarded_trials = get_binned_trials(dot_arr, N)
print('trt=',top_rewarded_trials.shape)
plot_avg_trajectories(fig, hs_pc_s, top_rewarded_trials, N)
# plot_trajectories(fig, hs_pc_s, top_rewarded_trials, N, T)


trt= (140, 8)


KeyboardInterrupt: 

In [46]:
# traj binned by dot speed

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]  # Omitting the last angle to avoid overlap with 0
    return np.column_stack((np.cos(angles), np.sin(angles)))

def compute_dot_speed(dot_arr):
    """Compute the average dot speed across timesteps for each trial."""
    speeds_per_timestep = np.linalg.norm(dot_arr, axis=2)
    avg_speeds_per_trial = np.mean(speeds_per_timestep, axis=1)
    return avg_speeds_per_trial

def bin_by_dot_speed(dot_speeds, sbins):
    bin_edges = np.linspace(dot_speeds.min(), dot_speeds.max(), sbins + 1)
    bin_indices = np.digitize(dot_speeds, bin_edges) - 1
    return bin_indices

def get_avg_trajectories(data, bin_indices, sbins):
    avg_trajectories = []
    for i in range(sbins):
        avg_trajectories.append(np.mean(data[bin_indices == i], axis=0))
    return avg_trajectories

def plot_avg_trajectories_by_speed(fig, data, dot_speeds, sbins):
    # Define a color wheel
    color_wheel = plt.cm.jet(np.linspace(0, 1, sbins))
    
    # Compute the binned indices
    bin_indices = bin_by_dot_speed(dot_speeds, sbins)

    # Calculate the average trajectories
    avg_trajectories = get_avg_trajectories(data, bin_indices, sbins)
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, avg_traj in enumerate(avg_trajectories): 
        segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
        lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
        
        ax1.add_collection(lc_3d)
        ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data.reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    # Create a legend based on speed bins
    bin_labels = ["Speed Bin " + str(i+1) for i in range(sbins)]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=bin_labels[i]) for i in range(sbins)]
    
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Average Trajectories by Dot Speed')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Average Trajectories by Dot Speed')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

sbins = 6
# Assuming hs_pc_s data and dot_arr are available

dot_speeds = compute_dot_speed(dot_arr)
fig = plt.figure(figsize=(12, 6))
plot_avg_trajectories_by_speed(fig, hs_pc_s, dot_speeds, sbins)


KeyboardInterrupt: 

In [48]:
# speed binned, single direction

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:,np.newaxis]
    return mean_dot_dir_normalized

def bin_by_dot_direction(dot_directions, bins):
    cosine_similarity = dot_directions @ np.array(bins).T
    bin_indices = np.argmax(cosine_similarity, axis=1)
    return bin_indices

def compute_dot_speed(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    speeds = np.linalg.norm(dot_diffs, axis=2)
    avg_speeds = np.mean(speeds, axis=1)
    return avg_speeds

def bin_by_speed(dot_speeds, sbins):
    bin_edges = np.linspace(dot_speeds.min(), dot_speeds.max(), sbins + 1)
    bin_indices = np.digitize(dot_speeds, bin_edges) - 1
    return bin_indices

def plot_avg_trajectories_by_direction_and_speed(fig, data, dot_directions, bins, dbin, dot_speeds, sbins):
    # Define a color wheel
    color_wheel = plt.cm.jet(np.linspace(0, 1, sbins))
    
    # Compute the binned indices for directions and speeds
    direction_indices = bin_by_dot_direction(dot_directions, bins)
    speed_indices = bin_by_speed(dot_speeds, sbins)

    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    # For the chosen direction bin, further bin by speed and plot averaged trajectories
    for speed_bin_idx in range(sbins):
        selected_indices = np.logical_and(direction_indices == dbin, speed_indices == speed_bin_idx)
        avg_traj = np.mean(data[selected_indices], axis=0)
        
        segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=color_wheel[speed_bin_idx])
        lc_2d = LineCollection(segments_2d, colors=color_wheel[speed_bin_idx])
        
        ax1.add_collection(lc_3d)
        ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data.reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    # Create a legend based on speed bins
    bin_labels = ["Speed Bin " + str(i+1) for i in range(sbins)]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=bin_labels[i]) for i in range(sbins)]
    
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title(f'3D Average Trajectories by Speed (Direction Bin {dbin})')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title(f'2D Projections of Average Trajectories by Speed (Direction Bin {dbin})')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

N = 8
bins = generate_bin_vectors(N)
dot_directions = compute_dot_direction(dot_arr)
dot_speeds = compute_dot_speed(dot_arr)
dbin = 1
sbins = 5
fig = plt.figure(figsize=(12, 6))
plot_avg_trajectories_by_direction_and_speed(fig, hs_pc_s, dot_directions, bins, dbin, dot_speeds, sbins)


KeyboardInterrupt: 

In [63]:
# speed, direction binned

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:,np.newaxis]
    return mean_dot_dir_normalized

def bin_by_dot_direction(dot_directions, bins):
    cosine_similarity = dot_directions @ np.array(bins).T
    bin_indices = np.argmax(cosine_similarity, axis=1)
    return bin_indices

def compute_dot_speed(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    speeds = np.linalg.norm(dot_diffs, axis=2)
    avg_speeds = np.mean(speeds, axis=1)
    return avg_speeds

def bin_by_speed(dot_speeds, sbins):
    bin_edges = np.linspace(dot_speeds.min(), dot_speeds.max(), sbins + 1)
    bin_indices = np.digitize(dot_speeds, bin_edges) - 1
    return bin_indices

def plot_avg_trajectories_by_all_directions(fig, data, dot_directions, bins, dot_speeds, sbins):
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    direction_indices = bin_by_dot_direction(dot_directions, bins)
    speed_indices = bin_by_speed(dot_speeds, sbins)

    # hsv colormap based on direction
    hue_values = np.linspace(0, 1, len(bins) + 1)[:-1]

    for dbin, hue in zip(range(len(bins)), hue_values):
        for speed_bin_idx in range(sbins):
            selected_indices = np.logical_and(direction_indices == dbin, speed_indices == speed_bin_idx)
            avg_traj = np.mean(data[selected_indices], axis=0)
            
            segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
            segments_2d = [s[:, :2] for s in segments_3d]
            
            # darker color for faster speed bins
            color = hsv_to_rgb((hue, 1, 1 - speed_bin_idx / (sbins + 1)))
            
            lc_3d = Line3DCollection(segments_3d, colors=color)
            lc_2d = LineCollection(segments_2d, colors=color)
            
            ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
            
            # optional, plot grey dot at the 15th timestep
            # ax1.scatter(avg_traj[14, 0], avg_traj[14, 1], avg_traj[14, 2], color='grey', s=30)
            # ax2.plot(avg_traj[14, 0], avg_traj[14, 1], 'o', color='grey', markersize=4)

    # Set limits
    all_data = data.reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())

    ax1.set_title('3D Average Trajectories by Direction and Speed')
    ax2.set_title('2D Projections of Average Trajectories by Direction and Speed')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')

    # Legend on the ax2 subplot
    legend_elements = [Line2D([0], [0], color=hsv_to_rgb((hue, 1, 0.5)), lw=2, label=f"Direction Bin {dbin}") for dbin, hue in zip(range(len(bins)), hue_values)]
    ax2.legend(handles=legend_elements, loc="upper right")
    
    plt.tight_layout()
    plt.show()

N = 4
bins = generate_bin_vectors(N)
dot_directions = compute_dot_direction(dot_arr)
dot_speeds = compute_dot_speed(dot_arr)
sbins = 3

fig = plt.figure(figsize=(12, 6))  
plot_avg_trajectories_by_all_directions(fig, hs_pc_s, dot_directions, bins, dot_speeds, sbins)


KeyboardInterrupt: 

In [54]:
# speed, direction binned (old/doesnt work)

import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]
    return np.column_stack((np.cos(angles), np.sin(angles)))

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def compute_speed(dot_arr):
    diff = np.diff(dot_arr, axis=1)
    speed = np.linalg.norm(diff, axis=2)
    avg_speed = np.mean(speed, axis=1)
    return avg_speed

def get_binned_indices(dot_arr, dbins, sbins):
    bins = generate_bin_vectors(dbins)
    dot_dirs = compute_dot_direction(dot_arr)
    speeds = compute_speed(dot_arr)
    speed_bins = np.linspace(np.min(speeds), np.max(speeds), sbins+1)
    
    bin_indices = {}
    
    for i, dot in enumerate(dot_dirs):
        # Find dot vector bin
        cosine_similarity = dot @ bins.T
        d_bin = np.argmax(cosine_similarity)

        # Find speed bin
        s_bin = np.digitize(speeds[i], speed_bins) - 1

        if (d_bin, s_bin) not in bin_indices:
            bin_indices[(d_bin, s_bin)] = []
        bin_indices[(d_bin, s_bin)].append(i)
    
    return bin_indices

def plot_avg_trajectories_by_bins(fig, data, dot_arr, dbins, sbins):   
    avg_dot_vectors = np.mean(np.linalg.norm(dot_arr, axis=-1), axis=-1)
    d_bin_edges = np.linspace(avg_dot_vectors.min(), avg_dot_vectors.max(), dbins+1)

    # Calculate speed for each trial as the mean speed over timesteps
    speeds = np.linalg.norm(np.diff(data, axis=1), axis=2).mean(axis=1)

    bin_indices = defaultdict(list)
    s_bin_edges_list = []

    for d in range(dbins):
        d_mask = (dot_arr >= d_bin_edges[d]) & (dot_arr < d_bin_edges[d+1])
        
        if np.sum(d_mask) == 0:  # No data points in this dot bin
            s_bin_edges_list.append(np.linspace(0, 1, sbins+1))  # Dummy values
            continue
        
        relevant_speeds = speeds[d_mask]
        s_bin_edges = np.linspace(relevant_speeds.min(), relevant_speeds.max(), sbins+1)
        s_bin_edges_list.append(s_bin_edges)
        
        for s in range(sbins):
            s_mask = (speeds >= s_bin_edges[s]) & (speeds < s_bin_edges[s+1])
            combined_mask = d_mask & s_mask
            bin_indices[(d, s)].extend(np.where(combined_mask)[0])
            
    ax = fig.add_subplot(1, 1, 1)
    color_wheel = plt.cm.jet(np.linspace(0, 1, dbins*sbins))
    
    for idx, trials in enumerate(bin_indices.values()):
        avg_traj = np.mean(data[trials], axis=0)
        ax.plot(avg_traj[:, 0], avg_traj[:, 1], color=color_wheel[idx])
        
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Averaged Trajectories Binned by Dot Vector and Speed')
    plt.show()

dbins = 8
sbins = 2

# Usage example (assuming you've loaded or generated 'dot_arr' and 'hs_pc_s'):
fig = plt.figure(figsize=(12, 6))
plot_avg_trajectories_by_bins(fig, hs_pc_s, dot_arr, dbins, sbins)


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [49]:
# speed, direction binned (old/doesnt work)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def compute_dot_speed(dot_arr):
    """Compute the average dot speed across timesteps for each trial."""
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_vel = np.mean(dot_diffs, axis=1)
    mean_dot_speeds = np.linalg.norm(mean_dot_vel, axis=1)
    return mean_dot_speeds

def bin_by_direction(dot_directions, dbins):
    bin_edges = np.linspace(-1, 1, dbins + 1)  # Assuming directions range from -1 to 1
    bin_indices = np.digitize(dot_directions, bin_edges) - 1
    return bin_indices

def bin_by_speed(dot_speeds, sbins):
    bin_edges = np.linspace(dot_speeds.min(), dot_speeds.max() + 1e-10, sbins + 1)
    bin_indices = np.digitize(dot_speeds, bin_edges) - 1
    return bin_indices

def plot_avg_trajectories_by_direction_and_speed(fig, data, dot_directions_bins, dbins, dot_speeds, sbins):
    # Define primary colors for the direction bins
    primary_colors = plt.cm.tab10(np.linspace(0, 1, dbins))
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    legend_elements = []
    
    for dir_bin_idx in range(dbins):
        # Get shades for speed bins within the primary color
        shades = [primary_colors[dir_bin_idx] * (0.5 + 0.5 * (i+1)/sbins) for i in range(sbins)]
        
        # Create legend for current direction bin
        for speed_bin_idx in range(sbins):
            legend_elements.append(Line2D([0], [0], color=shades[speed_bin_idx], lw=2, 
                                        label=f"Direction {dir_bin_idx + 1}, Speed Bin {speed_bin_idx + 1}"))
        
        for speed_bin_idx in range(sbins):
            selected_indices = np.logical_and(dot_directions_bins[:, 0] == dir_bin_idx, 
                                            bin_by_speed(dot_speeds, sbins) == speed_bin_idx)
            
            # Debugging print statement
            print(f"Direction {dir_bin_idx + 1}, Speed Bin {speed_bin_idx + 1}: {selected_indices.sum()} selected trials")
            
            avg_traj = np.mean(data[selected_indices], axis=0)
            
            # Debugging print statement
            if np.isnan(avg_traj).any():
                print(f"Direction {dir_bin_idx + 1}, Speed Bin {speed_bin_idx + 1} has NaN values in the averaged trajectory.")
            
            segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
            segments_2d = [s[:, :2] for s in segments_3d]
            
            lc_3d = Line3DCollection(segments_3d, colors=shades[speed_bin_idx])
            lc_2d = LineCollection(segments_2d, colors=shades[speed_bin_idx])
            
            ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data.reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Average Trajectories by Direction and Speed')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Average Trajectories by Direction and Speed')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.tight_layout()
    plt.show()

# Assuming your bins and hs_pc_s data are available
dbins = 8
sbins = 2

dot_directions = compute_dot_direction(dot_arr)
dot_directions_bins = bin_by_direction(dot_directions, dbins)
dot_speeds = compute_dot_speed(dot_arr)
fig = plt.figure(figsize=(14, 7))
plot_avg_trajectories_by_direction_and_speed(fig, hs_pc_s, dot_directions_bins, dbins, dot_speeds, sbins)


Direction 1, Speed Bin 1: 147 selected trials
Direction 1, Speed Bin 2: 92 selected trials
Direction 2, Speed Bin 1: 61 selected trials
Direction 2, Speed Bin 2: 43 selected trials
Direction 3, Speed Bin 1: 55 selected trials
Direction 3, Speed Bin 2: 33 selected trials
Direction 4, Speed Bin 1: 47 selected trials
Direction 4, Speed Bin 2: 17 selected trials
Direction 5, Speed Bin 1: 57 selected trials
Direction 5, Speed Bin 2: 33 selected trials
Direction 6, Speed Bin 1: 58 selected trials
Direction 6, Speed Bin 2: 22 selected trials
Direction 7, Speed Bin 1: 63 selected trials
Direction 7, Speed Bin 2: 41 selected trials
Direction 8, Speed Bin 1: 152 selected trials
Direction 8, Speed Bin 2: 79 selected trials


KeyboardInterrupt: 

In [36]:
# debug (hist)

import numpy as np
import matplotlib.pyplot as plt

# Assuming your dot_arr is defined and normalized
norms = np.linalg.norm(dot_arr, axis=2)
normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]

# Compute the angles of these vectors in degrees
angles = np.arctan2(normalized_dot_arr[:,:,1], normalized_dot_arr[:,:,0]) 
angles_deg = np.degrees(angles) % 360  # Convert to degrees and ensure the range is [0, 360)

# Flatten the angles array for the histogram
angles_flat = angles_deg.flatten()

# Plot histogram
plt.figure(figsize=(10,6))
plt.hist(angles_flat, bins=360, range=(0,360), color='blue', alpha=0.7)
plt.title("Histogram of Directions of Normalized Dot Vectors")
plt.xlabel("Direction (degrees)")
plt.ylabel("Number of Dot Vectors")
plt.grid(True)
plt.show()


KeyboardInterrupt: 

In [67]:
# debug (scatter)

import numpy as np
import matplotlib.pyplot as plt

# Assuming your bins are defined elsewhere
bins = generate_bin_vectors(N)

# Calculate the average dot velocity vectors
dot_diffs = np.diff(dot_arr, axis=1)
mean_dot_vel = np.mean(dot_diffs, axis=1)
norm_dot_vel = np.linalg.norm(mean_dot_vel, axis=1)
normalized_dot_vel = mean_dot_vel / norm_dot_vel[:, np.newaxis]

# Assign each normalized dot velocity vector to a bin
bin_assignments = []
for dot in normalized_dot_vel:
    cosine_similarity = dot @ np.array(bins).T
    assigned_bin = np.argmax(cosine_similarity)
    bin_assignments.append(assigned_bin)

# Scatter plot each bin
plt.figure(figsize=(10,10))
for idx, bin_val in enumerate(bins):
    selected_dots = normalized_dot_vel[np.array(bin_assignments) == idx] # normalized_dot_vel[np.array(bin_assignments) == idx]
    print('idx=',idx,'selected_dots.shape=',selected_dots.shape)
    plt.scatter(selected_dots[:, 0], selected_dots[:, 1], label=f'Bin {idx}')

plt.legend()
plt.title("Spread of normalized dot velocity vectors for each bin")
plt.xlabel("X-coordinate")
plt.ylabel("Y-coordinate")
plt.grid(True)
plt.show()

print('bin_assignments=',bin_assignments)

idx= 0 selected_dots.shape= (254, 2)
idx= 1 selected_dots.shape= (236, 2)
idx= 2 selected_dots.shape= (260, 2)
idx= 3 selected_dots.shape= (250, 2)


KeyboardInterrupt: 

In [7]:
# debug (scatter 2)

import numpy as np
import matplotlib.pyplot as plt

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]  # Omitting the last angle to avoid overlap with 0
    return np.column_stack((np.cos(angles), np.sin(angles)))

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def get_top_rewarded_trials_new(dot_arr, r_arr, T, N):
    bins = generate_bin_vectors(N)
    normalized_dot_dirs = compute_dot_direction(dot_arr)

    # Assign each dot to a bin
    bin_assignments = []
    for dot in normalized_dot_dirs:
        cosine_similarity = dot @ np.array(bins).T
        assigned_bin = np.argmax(cosine_similarity)
        bin_assignments.append(assigned_bin)

    # Organize trials by their assigned bin and then sort them by total reward
    total_rewards = np.sum(r_arr, axis=1)
    bin_lists = [[] for _ in range(N)]
    for i, assigned_bin in enumerate(bin_assignments):
        bin_lists[assigned_bin].append(i)
    
    # Sort trials in each bin by reward and select top T trials
    top_trials = np.zeros((T, N), dtype=int)
    for idx, trial_list in enumerate(bin_lists):
        sorted_trials = sorted(trial_list, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    return top_trials

def get_top_rewarded_trials_old(dot_arr, r_arr, T, N):
    norms = np.linalg.norm(dot_arr, axis=2)
    normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]
    bins = generate_bin_vectors(N)
    
    # Create an array to store the top K trials for each bin
    top_trials = np.empty((T, N), dtype=int)

    # Calculate the sum of rewards for each trial
    total_rewards = np.sum(r_arr, axis=1)

    for idx, bin_val in enumerate(bins):
        # Calculate the angle (cosine similarity) between each dot vector and the current bin vector
        cosine_similarity = normalized_dot_arr @ np.array(bin_val)
        
        # Convert cosine similarity to angular difference
        angular_diff = np.arccos(cosine_similarity)

        # Check if this bin has the smallest angular difference for each dot vector
        is_closest_bin = np.isclose(angular_diff, np.min(angular_diff, axis=1)[:, np.newaxis])
        
        # Find the top K trials for the current bin
        sorted_indices = np.argsort(total_rewards[is_closest_bin[:, 0]])[::-1]
        top_trials[:, idx] = np.arange(1000)[is_closest_bin[:, 0]][sorted_indices[:T]]

    return top_trials

def get_top_rewarded_trials(dot_arr, r_arr, T, N):
    norms = np.linalg.norm(dot_arr, axis=2)
    normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]
    bins = generate_bin_vectors(N)
    
    # Store the trials for each bin in a list of lists
    bin_trials = [[] for _ in range(N)]

    for trial_idx, dots in enumerate(normalized_dot_arr):
        # Calculate the angle (cosine similarity) between this trial's dot vector and all bins
        cosine_similarity = dots @ bins.T
        angular_diff = np.arccos(cosine_similarity)
        # Get the index of the bin that has the smallest angular difference for this trial
        closest_bin_idx = np.argmin(angular_diff, axis=1)[0] 
        bin_trials[closest_bin_idx].append(trial_idx)
        
    top_trials = np.empty((T, N), dtype=int)
    total_rewards = np.sum(r_arr, axis=1)

    for idx, trials in enumerate(bin_trials):
        # Sort the trials in this bin based on their rewards and take the top T
        sorted_trials = sorted(trials, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    t = has_duplicates(top_trials)
    print('has duplicates=',t)

    return top_trials

def plot_scatter_top_trials(dot_arr, top_trials, bins):
    # Calculate the average dot velocity vectors
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_vel = np.mean(dot_diffs, axis=1)
    
    plt.figure(figsize=(10,10))
    
    # Draw the dotted lines for bin edges
    max_radius = np.max(np.linalg.norm(mean_dot_vel, axis=1))
    for bin_vec in bins:
        end_x = max_radius * bin_vec[0]
        end_y = max_radius * bin_vec[1]
        plt.plot([0, end_x], [0, end_y], 'k--')
    
    # Scatter plot for each bin and compute the center of mass
    for idx, trials in enumerate(top_trials.T):
        selected_dots = mean_dot_vel[trials]
        plt.scatter(selected_dots[:, 0], selected_dots[:, 1], label=f'Bin {idx}', s=20, alpha=0.5)
        
        # Calculate and plot the center of mass for this bin
        centroid_x = np.mean(selected_dots[:, 0])
        centroid_y = np.mean(selected_dots[:, 1])
        plt.scatter(centroid_x, centroid_y, marker='x', s=200)
        
    plt.legend()
    plt.title("Scatter plot of dot velocities for top trials with centroids")
    plt.xlabel("X-coordinate")
    plt.ylabel("Y-coordinate")
    plt.grid(True)
    plt.show()

N = 8
T = 100

bins = generate_bin_vectors(N)  # Ensure you have the bins for your problem

# Generate top trials using the old function
top_trials = get_top_rewarded_trials_new(dot_arr, r_arr, T, N)
print('top_trials=',top_trials.shape)

# Plot the scatter plot using the generated top trials
plot_scatter_top_trials(dot_arr, top_trials, bins)


top_trials= (100, 8)


KeyboardInterrupt: 

In [ ]:
# debug (dot vel scatter)

%matplotlib inline
dot_diffs = np.diff(dot_arr, axis=1)
mean_dot_vel = np.mean(dot_diffs, axis=1)
norm_dot_vel = np.linalg.norm(mean_dot_vel, axis=1)
mean_dot_vel = mean_dot_vel / norm_dot_vel[:,np.newaxis]
print('dot_diffs.shape=',dot_diffs.shape)
print('mean_dot_vel.shape=',mean_dot_vel.shape)
print('norm_dot_vel.shape=',norm_dot_vel.shape)
print('mean_dot_vel=',mean_dot_vel.shape)
# plot a scatter plot of the dot velocities:
plt.figure(figsize=(10,6))
plt.scatter(mean_dot_vel[:,0], mean_dot_vel[:,1], s=5)
plt.title("Scatter Plot of Dot Velocities")
plt.xlabel("Dot Velocity in X")
plt.ylabel("Dot Velocity in Y")

In [ ]:
# get_top_rewarded_trials fncs (old)

def get_top_rewarded_trials_old(dot_arr, r_arr, T, N):
    norms = np.linalg.norm(dot_arr, axis=2)
    normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]
    bins = generate_bin_vectors(N)
    print('bins=',bins)
    
    # Create an array to store the top K trials for each bin
    top_trials = np.empty((T, N), dtype=int)

    # Calculate the sum of rewards for each trial
    total_rewards = np.sum(r_arr, axis=1)

    for idx, bin_val in enumerate(bins):
        print('bin_val=',bin_val)
        # Calculate the angle (cosine similarity) between each dot vector and the current bin vector
        cosine_similarity = normalized_dot_arr @ np.array(bin_val)
        
        # Convert cosine similarity to angular difference
        angular_diff = np.arccos(cosine_similarity)
        print('angular_diff=',angular_diff.shape)

        # print('min_ang_diff=',np.min(angular_diff, axis=1))

        # Check if this bin has the smallest angular difference for each dot vector
        is_closest_bin = np.isclose(angular_diff, np.min(angular_diff, axis=1)[:, np.newaxis])
        
        print(f"Number of trials closest to bin {idx}: {sum(is_closest_bin[:, 0])}")
        # Find the top K trials for the current bin
        sorted_indices = np.argsort(total_rewards[is_closest_bin[:, 0]])[::-1]
        top_trials[:, idx] = np.arange(1000)[is_closest_bin[:, 0]][sorted_indices[:T]]

    t = has_duplicates(top_trials)
    print('has duplicates=',t)

    return top_trials

def get_top_rewarded_trials(dot_arr, r_arr, T, N):
    norms = np.linalg.norm(dot_arr, axis=2)
    normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]
    bins = generate_bin_vectors(N)
    
    # Store the trials for each bin in a list of lists
    bin_trials = [[] for _ in range(N)]

    for trial_idx, dots in enumerate(normalized_dot_arr):
        # Calculate the angle (cosine similarity) between this trial's dot vector and all bins
        cosine_similarity = dots @ bins.T
        angular_diff = np.arccos(cosine_similarity)
        # Get the index of the bin that has the smallest angular difference for this trial
        closest_bin_idx = np.argmin(angular_diff, axis=1)[0] 
        bin_trials[closest_bin_idx].append(trial_idx)
        
    top_trials = np.empty((T, N), dtype=int)
    total_rewards = np.sum(r_arr, axis=1)

    for idx, trials in enumerate(bin_trials):
        # Sort the trials in this bin based on their rewards and take the top T
        sorted_trials = sorted(trials, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    t = has_duplicates(top_trials)
    print('has duplicates=',t)

    return top_trials

In [ ]:
# old

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import LineCollection

n = 335

def plot_trajectory(data, n):
    fig = plt.figure(figsize=(12, 6))
    
    # 3D Plot
    ax1 = fig.add_subplot(121, projection='3d')
    traj = data[n, :, :3]  # Considering top 3 PCs
    length = traj.shape[0]
    segments = [traj[i:i+2] for i in range(length-1)]
    lc_3d = Line3DCollection(segments, cmap=plt.cm.seismic, norm=plt.Normalize(0, length))
    lc_3d.set_array(np.linspace(0, length, len(segments)))
    ax1.add_collection(lc_3d)
    ax1.set_xlim(traj[:, 0].min(), traj[:, 0].max())
    ax1.set_ylim(traj[:, 1].min(), traj[:, 1].max())
    ax1.set_zlim(traj[:, 2].min(), traj[:, 2].max())
    ax1.set_title('3D Trajectory')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')

    # 2D Plot (assuming the plane of most variation involves the first and second PCs)
    ax2 = fig.add_subplot(122)
    segments_2d = [traj[i:i+2, :2] for i in range(length-1)]
    lc_2d = LineCollection(segments_2d, cmap=plt.cm.seismic, norm=plt.Normalize(0, length))
    lc_2d.set_array(np.linspace(0, length, len(segments_2d)))
    ax2.add_collection(lc_2d)
    ax2.autoscale_view()
    ax2.set_title('2D Projection')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')

    plt.tight_layout()
    plt.show()
    plt.close()

plot_trajectory(hs_pc_s, n)
# plot_trajectory(hv_pc_c, n)

